In [1]:
import json
from pprint import pprint
from matplotlib import pyplot as plt
import cv2
import copy
import operator
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, ZeroPadding2D, Dropout
from keras.callbacks import TensorBoard
from keras.models import load_model
from keras.optimizers import Adam
from time import time
import os
%matplotlib qt

Using TensorFlow backend.


In [2]:
def readJson(file):
    with open(file, 'r') as fp:
        data = json.load(fp)
        
    return data

In [3]:
def read_jsons(folder):
    jsons = [json for json in os.listdir(folder) if json.endswith(".json")]
    
    return jsons

In [4]:
def data_points(jsons, json_folder_dir):
    data_points = []
    
    for i in range(0, len(jsons)):
        data_points.append([])
        with open((json_folder_dir + jsons[i])) as f:
            data = json.load(f)
        
        n_people = len(data['people'])
    
        for j in range(0, n_people):
            data_points[i].append([])
            datas = data['people'][j]['pose_keypoints_2d']
        
            for k in range(0, len(datas)):
                data_points[i][j].append(datas[k])
                
    return data_points

In [5]:
def data_classification(data):
    body_label = {0:"Nose", 1:"Neck", 2:"RShoulder", 3:"RElbow", 4:"RWrist", 5:"LShoulder", 6:"LElbow", 7:"LWrist", 8:"MidHip", 9:"RHip", 10:"RKnee", 11:"RAnkle", 12:"LHip", 13:"LKnee",14:"LAnkle", 15:"REye", 16:"LEye", 17:"REar", 18:"LEar", 19:"LBigToe", 20:"LSmallToe", 21:"LHeel", 22:"RBigToe", 23:"RSmallToe", 24:"RHeel"}
    parts = {}
    people = {}
    
    for person in range(0, len(data)):
        points = data[person]
        for i in range(0, len(points)):
            parts[body_label[i]] = points[i]
        
        people[person] = parts
        parts = {}
        
    return people

In [6]:
def organise_data(data, number):
    total = len(data)
    people_body_point = {}
    
    for i in range(0, total):
        people = len(data[i])
        frame_points = {}
        id_number = 0
        if people != 0:
            for j in range(0, people):
                points = data[i][j]
                n_points = len(points)
                body_points = []
                data_certainty = 0
                for k in range(0, n_points - 1, 3):
                    body_points.append((points[0 + k], points[1 + k]))
                    data_certainty += points[2 + k]
                    
                
                data_certainty = data_certainty / number
                
                if data_certainty >= 0.5:
                    frame_points[id_number] = body_points
                    id_number += 1
                
            organised = data_classification(frame_points)
            people_body_point[i] = organised
                
    return people_body_point

In [7]:
def reformat(data):
    
    exercise = {}
    
    for i in data:
        for j in data[i]:
            exercise[i] = data[i][j]
            
    return exercise

In [8]:
def section_separator(data, start, end):
    
    exercise_one = {}
    exercise_two = {}
    
                
    for i in range(start, end + 1):
        
        for j in data[str(i)]:
            if j == '0':
                exercise_one[i] = data[str(i)][j]
            else:
                exercise_two[i] = data[str(i)][j]
        
#         i += 1

    return exercise_one, exercise_two

In [9]:
def organiseAllPoints(data):
    new_data = {}
    
    keys = ['Nose', 'Neck', 'RShoulder', 'RElbow', 'RWrist', 'LShoulder', 'LElbow', 'LWrist', 'MidHip', 'RHip', 'RKnee', 'RAnkle', 'LHip', 'LKnee', 'LAnkle', 'REye', 'LEye', 'REar', 'LEar', 'LBigToe', 'LSmallToe', 'LHeel', 'RBigToe', 'RSmallToe', 'RHeel']
    
    # Initialisation
    for i in range(len(keys)):
        
        new_data[keys[i]] = []
       
    for keys in data:
        for Lkeys in data[keys]:
            coordinate = new_data[Lkeys]
            
            points = data[keys][Lkeys]
            
            if points[0] != 0 and points[1] != 0:
                previousPoints = copy.deepcopy(points)
                coordinate.append(points)
            else:
                coordinate.append(previousPoints)
            
#             coordinate.append(points)
            
            new_data[Lkeys] = coordinate            
    
    return new_data

In [10]:
def rescale(data):
    
    allPoints = organiseAllPoints(data)
        
    new_data = {}
    
    for keys in allPoints:
#         ini = {}
#         ini['x'] = []
#         ini['y'] = []
        
        new_data[keys] = []
        
    fit_points = []
    first = True
    
    for keys in allPoints:
        if first:
            fit_points = allPoints[keys]
            first = False
        else:
            fit_points + allPoints[keys]
            
    scaler = StandardScaler().fit(fit_points)
    
    for keys in allPoints:
        points = scaler.transform(allPoints[keys])
        
#         x = points[:, 0]
#         y = points[:, 1]
#         x_mean = np.mean(x)
#         x_std = np.std(x)
#         y_mean = np.mean(y)
#         y_std = np.std(y)
        
#         X = [x_mean, x_std]
#         Y = [y_mean, y_std]
        
#         new_data[keys]['x'] = X
#         new_data[keys]['y'] = Y

        new_data[keys] = points
        
    return new_data

In [11]:
def merge(master, slave):
    original = copy.deepcopy(master)
    for i in master:
        masterList = master[i]
        slaveList = slave[i]
        
        finalList = np.concatenate((masterList, slaveList), axis = 0)
            
        original[i] = finalList
        
    return original

In [12]:
def returnNumber(superFolder, subFolder):
    number = 25
    
    if superFolder == 'Star_Jumps':
        if subFolder == 'Starjump10':
            number = 26
        if subFolder == 'Starjump7' or 'Starjump8' or 'Starjump9':
            number = 20
        
    if superFolder == 'Squats':
        number = 25

    if superFolder == 'Biceps':
        if subFolder == 'Bicep8' or subFolder == 'Bicep9' or subFolder == 'Bicep10':
            number = 20

    if superFolder == 'Chests':
        if subFolder == 'Chest6' or subFolder == 'Chest7' or subFolder == 'Chest8':
            number = 20

        if subFolder == 'Chest12':
            number = 15

    if superFolder == 'Punches':
        number = 20
        
    return number

In [13]:
def read_folders(superFolder):
    subFolders = next(os.walk('Sports_Skeleton/' + superFolder + '/.'))[1]
    
    keyStart = 0
    
    total = {}
    
    first = True
    
    for i in range(len(subFolders)):
        json_folder = 'Sports_Skeleton/' + superFolder + '/' + subFolders[i]
        
        number = returnNumber(superFolder, subFolders[i]) 
            
        json_folder_slash = json_folder + '/'
        
        jsons = read_jsons(json_folder)
        
        data_points1 = data_points(jsons, json_folder_slash)
        
        part_points = organise_data(data_points1, number)
                
        reformat_points = reformat(part_points)
        
        rescale_points = rescale(reformat_points)
        
        if first:
            total = rescale_points
            first = False
        else:
            for i in rescale_points:
                total = merge(total, rescale_points)
                                
    return total

In [14]:
def dicToList(data):
    bigList = []
    first = True
    for i in data:
        if first:
            bigList = data[i]
            first = False
        else:
             bigList = np.concatenate((bigList, data[i]), axis = 0)
    
    return bigList

In [15]:
def findmaximumpeople(json_folder_dir, jsons, id0, idn):
    
    if not id0 == idn:
        midval = (id0 + idn)//2
        leftval = findmaximumpeople(json_folder_dir, jsons, id0, midval)
        rightval = findmaximumpeople(json_folder_dir, jsons, midval + 1, idn)
        
        if leftval >= rightval:
            return leftval
        else:
            return rightval
    else:
        with open((json_folder_dir + jsons[id0])) as f:
            data = json.load(f)
        
        number = 0
        
        for item in data['people']:
            data_points = item['pose_keypoints_2d']
            
            total_certainty = 0
            
            n_points = len(data_points)
            
            division = 0
            
            for k in range(0, n_points - 1, 3):
                total_certainty += data_points[2 + k]
                division += 1
                
            total_certainty = total_certainty / 20
        
            if (total_certainty >= 0.5):
                number += 1

                
        return number
    

In [16]:
# section_player("DS3.MOV", ds4_data, 3580, 4370)
# 210, 980
# 1430, 2210
#2550, 3330
#3580, 4370

In [17]:
json_folder = 'Sports_Skeleton/Punches/Punches13'
json_folder_slash = json_folder + '/'

jsons = read_jsons(json_folder)

number_of_people = findmaximumpeople(json_folder_slash, jsons, 0, (len(jsons) - 1))

In [18]:
number_of_people

1

In [19]:
squats0 = read_folders('Squats')

In [20]:
arm0 = read_folders('Biceps')

In [21]:
starJump0 = read_folders('Star_Jumps')

In [22]:
chest0 = read_folders('Chests')

In [23]:
punch0 = read_folders('Punches')

In [24]:
ds2_data = readJson('Re-identified jsons/DS1.json')
ds3_data = readJson('Re-identified jsons/DS2.json')
ds4_data = readJson('Re-identified jsons/DS3.json')

In [25]:
squat1, arm1 = section_separator(ds2_data, 135, 800)
arm2, squat2 = section_separator(ds2_data, 1230, 1910)
squat3, arm3 = section_separator(ds2_data, 2700, 3350)
arm4, squat4 = section_separator(ds2_data, 3600, 4280)

In [26]:
squat5, arm5 = section_separator(ds3_data, 210, 930)
arm6, squat6 = section_separator(ds3_data, 1250, 1990)
squat7, arm7 = section_separator(ds3_data, 2330, 2980)
arm8, squat8 = section_separator(ds3_data, 3725, 4350)
squat9, arm9 = section_separator(ds3_data, 4585, 5325)

In [27]:
squat10, starJump1 = section_separator(ds4_data, 210, 980)
starJump2, squat11 = section_separator(ds4_data, 1430, 2210)
squat12, starJump3 = section_separator(ds4_data, 2550, 3330)
starJump4, squat13 = section_separator(ds4_data, 3580, 4370)

In [28]:
arms_rescale = {}
squats_rescale = {}
starJumps_rescale = {}

In [29]:
arms_rescale[0] = rescale(arm1)
arms_rescale[1] = rescale(arm2)
arms_rescale[2] = rescale(arm3)
arms_rescale[3] = rescale(arm4)
arms_rescale[4] = rescale(arm5)
arms_rescale[5] = rescale(arm6)
arms_rescale[6] = rescale(arm7)
arms_rescale[7] = rescale(arm8)
arms_rescale[8] = rescale(arm9)
squats_rescale[0] = rescale(squat1)
squats_rescale[1] = rescale(squat2)
squats_rescale[2] = rescale(squat3)
squats_rescale[3] = rescale(squat4)
squats_rescale[4] = rescale(squat5)
squats_rescale[5] = rescale(squat6)
squats_rescale[6] = rescale(squat7)
squats_rescale[7] = rescale(squat8)
squats_rescale[8] = rescale(squat9)
squats_rescale[9] = rescale(squat10)
squats_rescale[10] = rescale(squat11)
squats_rescale[11] = rescale(squat12)
squats_rescale[12] = rescale(squat13)
starJumps_rescale[0] = rescale(starJump1)
starJumps_rescale[1] = rescale(starJump2)
starJumps_rescale[2] = rescale(starJump3)
starJumps_rescale[3] = rescale(starJump4)

In [30]:
for i in range(0, 9):
    arm0 = merge(arm0, arms_rescale[i])

In [31]:
for i in range(0, 13):
    squats0 = merge(squats0, squats_rescale[i])

In [32]:
for i in range(0, 4):
    starJump0 = merge(starJump0, starJumps_rescale[i])

In [33]:
arm_size = len(arm0[list(arm0.keys())[0]])

In [34]:
squat_size = len(squats0[list(squats0.keys())[0]])

In [35]:
starJump_size = len(starJump0[list(starJump0.keys())[0]])

In [36]:
chest_size = len(chest0[list(chest0.keys())[0]])

In [37]:
punch_size = len(punch0[list(punch0.keys())[0]])

In [38]:
arms = dicToList(arm0)

In [39]:
squats = dicToList(squats0)

In [40]:
starJumps = dicToList(starJump0)

In [41]:
chests = dicToList(chest0)

In [42]:
punches = dicToList(punch0)

In [43]:
print(arms.shape)
print(squats.shape)
print(starJumps.shape)
print(chests.shape)
print(punches.shape)

(18219000, 2)
(15632525, 2)
(27334875, 2)
(15164125, 2)
(18871150, 2)


In [44]:
arms_shape = arms.reshape(arm_size, 50)
print(arms_shape.shape)

(728760, 50)


In [45]:
squats_shape = squats.reshape(squat_size, 50)
print(squats_shape.shape)

(625301, 50)


In [46]:
starJumps_shape = starJumps.reshape(starJump_size, 50)
print(starJumps_shape.shape)

(1093395, 50)


In [47]:
chests_shape = chests.reshape(chest_size, 50)
print(chests_shape.shape)

(606565, 50)


In [48]:
punches_shape = punches.reshape(punch_size,50)
print(punches_shape.shape)

(754846, 50)


In [49]:
if len(arms_shape) % 30 != 0:
    arms_shape = arms_shape[:-(len(arms_shape) % 30), :]

if len(squats_shape) % 30 != 0:
    squats_shape = squats_shape[:-(len(squats_shape) % 30), :]
    
if len(starJumps_shape) % 30 != 0:
    starJumps_shape = starJumps_shape[:-(len(starJumps_shape) % 30), :]

if len(chests_shape) % 30 != 0:
    chests_shape = chests_shape[:-(len(chests_shape) % 30), :]

if len(punches_shape) % 30 != 0:
    punches_shape = punches_shape[:-(len(punches_shape) % 30), :]

print(arms_shape.shape)
print(squats_shape.shape)
print(starJumps_shape.shape)
print(chests_shape.shape)
print(punches_shape.shape)

(728760, 50)
(625290, 50)
(1093380, 50)
(606540, 50)
(754830, 50)


In [50]:
final_arm = arms_shape.reshape(int(len(arms_shape) / 30), 30, 50, 1)
print(final_arm.shape)

(24292, 30, 50, 1)


In [51]:
final_squats = squats_shape.reshape(int(len(squats_shape)/30), 30, 50, 1)
print(final_squats.shape)

(20843, 30, 50, 1)


In [52]:
final_starJumps = starJumps_shape.reshape(int(len(starJumps_shape)/30), 30, 50, 1)
print(final_starJumps.shape)

(36446, 30, 50, 1)


In [53]:
final_chests = chests_shape.reshape(int(len(chests_shape)/30), 30, 50, 1)
print(final_chests.shape) 

(20218, 30, 50, 1)


In [54]:
final_punches = punches_shape.reshape(int(len(punches_shape)/30), 30, 50, 1)
print(final_punches.shape)

(25161, 30, 50, 1)


In [55]:
label = np.array([])

for i in range(int(len(arms_shape) / 30)):
    label = np.append(label, 0)

for i in range(int(len(squats_shape)/30)):
    label = np.append(label, 1)
    
for i in range(int(len(starJumps_shape)/30)):
    label = np.append(label, 2)
    
for i in range(int(len(chests_shape)/30)):
    label = np.append(label, 3)

for i in range(int(len(punches_shape)/30)):
    label = np.append(label, 4)
    
print(label.shape)
    

(126960,)


In [56]:
x_data = np.concatenate((final_arm, final_squats, final_starJumps, final_chests, final_punches))

In [57]:
x_data.shape

(126960, 30, 50, 1)

In [58]:
label = label.reshape((int(len(arms_shape) / 30)) + (int(len(squats_shape) / 30)) + (int(len(starJumps_shape) / 30)) + (int(len(chests_shape)/30)) + (int(len(punches_shape)/30)), 1)

In [59]:
label.shape

(126960, 1)

In [60]:
X_train, X_test, y_train, y_test = train_test_split(x_data, label, test_size = 0.3, random_state = 101 )

In [61]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [62]:
X_train.shape

(88872, 30, 50, 1)

In [63]:
model = Sequential()
model.add(Conv2D(64, kernel_size = 5, activation = 'relu', input_shape = (30, 50, 1)))

model.add(ZeroPadding2D((1,1)))
model.add(Dropout(0.2))
model.add(Conv2D(32, kernel_size = 4, activation = 'relu'))

model.add(ZeroPadding2D((1,1)))
model.add(Dropout(0.2))
model.add(Conv2D(16, kernel_size = 3, activation = 'relu'))

model.add(ZeroPadding2D((1,1)))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size = (8,8), strides = (3,3)))
model.add(Flatten())
model.add(Dense(5, activation='softmax'))

tensorboard = TensorBoard(log_dir='log/{}'.format(time()))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [64]:
model.compile(optimizer = Adam(lr = 0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [65]:
model.fit(X_train, y_train, validation_data =(X_test, y_test), epochs=100, callbacks=[tensorboard], verbose=False)

Instructions for updating:
Use tf.cast instead.


In [66]:
model.save('model/Exercise_Classification_Pooling_Dropout_0.2.h5')

In [67]:
del model